In [ ]:
import numpy as np
import datajoint as dj
import trimesh

In [ ]:
# testing which table to pull from
pinky = dj.create_virtual_module("pinky","microns_pinky")
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [ ]:
#gets the possible segment ids
segment_ids = (pinky.AllenSoma() & "cell_class='excitatory'").fetch("segment_id")

In [ ]:
(pinky.Mesh & key).fetch("vertices","triangles")

In [ ]:
"""
Brainstorming what to have in the table:
n_vertices
n_triangles
vertices
traingels
largest_piece_perc
n_pieces
outside_perc
stitched_addon_perc
unstitched_perce



"""

In [ ]:
ta3p100.Or

In [ ]:
pinky.Mesh()

In [1]:
from Stitcher import stitch_neuron

In [3]:
from Stitcher import stitch_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time


use_datajoint = True
segment_id = 648518346349470171
stitch_neuron_start = time.time()

#getting data from datajoint
pinky = dj.create_virtual_module("pinky","microns_pinky")


print("About to download mesh data")
key = dict(segment_id=segment_id,segmentation=3)
vertices,triangles = (pinky.Mesh & key).fetch("vertices","triangles")

verts = vertices[0]
faces = triangles[0]

print("About to run stitch_neuron")

[n_vertices,
 n_triangles,
 vertices,
 triangles,
 n_pieces,
 largest_piece_perc,
 outside_perc,
 n_stitched,
 stitched_addon_perc,
 n_unstitched,
 unstitched_perce] = stitch_neuron(segment_id=segment_id,
                                              vertices=verts,
                                              faces=faces,
                                            import_from_off=False,
                                             load_meshes_flag = False,
                                         save_meshes_flag = True,
                                            pymeshfix_flag = False
                                           )

#print the results:
[n_vertices,
     n_triangles,
     vertices,
     triangles,
     n_pieces,
     largest_piece_perc,
     outside_perc,
     n_stitched,
     stitched_addon_perc,
     n_unstitched,
     unstitched_perce]

Connecting celiib@10.28.0.34:3306
About to download mesh data
About to run stitch_neuron
generating child and main meshes
There were 3902 pieces after mesh split
There were 378 pieces found after size threshold
max_index = 0
max_face_len = 1845336
Total time for Mesh Cleansing: 261.9530200958252
Number of outside child meshes =  43
Main mesh and outside child meshes make up 0.8810968024950274% of original mesh
Saved child and main meshes at stitch_mesh_saved/648518346349470171_30_main_and_child_meshes_array.npz
 face length 1845336 using optimized facets with 3 neighbors
Filtering edges by size finished: 22.473762273788452, facet # = 96935
Filtering by convexity and getting centers took: 32.05838871002197, facet # = 19837
Finished facets for main mesh: 56.93193197250366 with facet length = 19837
Finished 19837 facets for main mesh: 56.93198823928833
Filtering edges by size finished: 1.5272104740142822, facet # = 6084
Filtering by convexity and getting centers took: 1.9699411392211914, 

[1102811,
 2198140,
 TrackedArray([[458752.        , 236544.        ,  56220.        ],
               [458752.        , 236544.        ,  56140.        ],
               [458752.        , 164608.        ,  55540.        ],
               ...,
               [464758.90625   , 243382.90625   ,  33091.21484375],
               [464787.96875   , 243365.046875  ,  32867.328125  ],
               [464563.96875   , 243447.875     ,  33007.4609375 ]]),
 TrackedArray([[ 479845,  538425,  791492],
               [ 462027,  759342,  847848],
               [ 293562,  502304,  191025],
               ...,
               [1102803, 1102788, 1102665],
               [1102730, 1102803, 1102665],
               [1102795, 1102803, 1102730]]),
 43,
 0.73926843778356,
 0.8810968024950274,
 42,
 0.19118686244673058,
 1,
 1.7341015403156932e-05]

In [ ]:
@schema
class ExcitatoryStitchedMesh(dj.Computed):
    definition="""
    -> pinky.Mesh
    ---
    n_vertices           : bigint           # number of vertices in this mesh
    n_triangles          : bigint           # number of triangles in this mesh
    vertices             : longblob         # x,y,z coordinates of vertices
    triangles            : longblob         # triangles (triplets of vertices)
    n_pieces             : int              # number of unconnected mesh pieces outside the largest mesh piece
    largest_piece_perc   : decimal(6,5)     # number of faces percentage of largest mesh piece in respect to total mesh
    outside_perc         : decimal(6,5)     # number of faces percentage of mesh outside the biggest mesh piece
    n_stitched           : int              # number of mesh pieces stitched back to main mesh
    stitched_addon_perc  : decimal(6,5)     # number of faces percentage of pieces that were stitched back in respect to largest mesh piece
    n_unstitched         : int              # number of mesh pieces remaining unstitched back to main mesh        
    unstitched_perce     : decimal(6,5)     # number of faces percentage of pieces that were not in respect to largest mesh piece
    
    
    """
    
    key_source = ((dj.U("segmentation","segment_id") & ta3p100.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & ta3p100.CoarseLabelOrphan.proj()))
    
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        
        
        
        self.insert1(dict(key,
                         ))